In [8]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [10]:
tf.keras.backend.clear_session()

In [11]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [12]:
subject = 'Paraná - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [13]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_PR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Paraná - Produção de Cimento (t)',axis=1)
data

,Ano,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Consumo de Cimento (t)
0,2008,8.033948,0.782138,0.768548,7.755114e+06,422290.202070,9.424555,6.724366e+06,0.745648,0.834122,...,2.868108,11.760983,17.558665,912.624708,1226.296848,18.012164,13.653739,78.027482,2.236583,3001.521000
1,2009,8.063370,0.782910,0.767137,7.805166e+06,424173.603186,9.409042,6.722105e+06,0.748601,0.835310,...,2.305885,9.472585,16.652522,912.466179,1217.653383,17.954863,13.668549,78.238643,2.155189,3092.068000
2,2010,8.026843,0.783594,0.766111,7.991788e+06,416829.675918,9.394384,6.762279e+06,0.751387,0.835940,...,1.958246,9.332485,17.928528,912.482023,1194.060192,17.703414,13.001948,76.311573,2.092804,3749.448000
3,2011,8.013806,0.784190,0.765458,8.164220e+06,410997.875281,9.386906,6.795421e+06,0.753998,0.836020,...,2.079088,10.985689,18.661309,912.629463,1182.416452,17.710041,12.630186,74.615878,2.043995,4174.942000
4,2012,6.472496,0.784389,0.764921,8.325627e+06,407346.950074,9.383420,6.827686e+06,0.755977,0.835445,...,2.183282,8.159014,19.310061,912.778766,1176.491125,17.874291,12.612094,73.865245,1.941425,4522.969000
5,2013,7.011799,0.785421,0.765581,8.462722e+06,403688.428174,9.371494,6.849508e+06,0.759142,0.834565,...,2.403445,7.913561,19.233593,912.914618,1170.878793,18.105264,12.720105,73.487865,1.844730,4578.716000
6,2014,7.462443,0.785706,0.765963,8.560896e+06,400704.121188,9.343267,6.852462e+06,0.760987,0.833112,...,2.748397,10.350442,20.725057,912.973266,1150.562966,18.275455,13.157699,74.605015,1.851460,4341.821857
7,2015,7.813261,0.785515,0.766043,8.597116e+06,391942.316578,9.284504,6.825760e+06,0.762152,0.831122,...,2.501529,12.454398,22.054778,913.057535,1145.712487,18.177724,13.346348,75.170150,1.905966,4108.786286
8,2016,7.840669,0.788366,0.758774,8.620855e+06,389049.090551,9.247193,6.809907e+06,0.771114,0.838386,...,2.073464,13.111908,22.315526,913.132268,1158.573328,18.115048,13.261122,74.043778,1.933375,3869.334857
9,2017,7.553100,0.787582,0.759026,8.632052e+06,389052.079088,9.235469,6.808242e+06,0.770943,0.835784,...,2.177386,9.479812,21.977949,913.165670,1160.957791,18.281050,13.470416,73.546507,1.889856,3764.518000


In [14]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,8.033948,0.782138,0.768548,7.755114e+06,422290.202070,9.424555,6.724366e+06,0.745648,0.834122,5.483335e+08,...,-7723.574973,2.868108,11.760983,17.558665,912.624708,1226.296848,18.012164,13.653739,78.027482,2.236583
1,8.063370,0.782910,0.767137,7.805166e+06,424173.603186,9.409042,6.722105e+06,0.748601,0.835310,5.302764e+08,...,-8961.781709,2.305885,9.472585,16.652522,912.466179,1217.653383,17.954863,13.668549,78.238643,2.155189
2,8.026843,0.783594,0.766111,7.991788e+06,416829.675918,9.394384,6.762279e+06,0.751387,0.835940,5.046425e+08,...,-9895.682385,1.958246,9.332485,17.928528,912.482023,1194.060192,17.703414,13.001948,76.311573,2.092804
3,8.013806,0.784190,0.765458,8.164220e+06,410997.875281,9.386906,6.795421e+06,0.753998,0.836020,4.854622e+08,...,-9989.170714,2.079088,10.985689,18.661309,912.629463,1182.416452,17.710041,12.630186,74.615878,2.043995
4,6.472496,0.784389,0.764921,8.325627e+06,407346.950074,9.383420,6.827686e+06,0.755977,0.835445,4.867661e+08,...,-9429.326328,2.183282,8.159014,19.310061,912.778766,1176.491125,17.874291,12.612094,73.865245,1.941425
5,7.011799,0.785421,0.765581,8.462722e+06,403688.428174,9.371494,6.849508e+06,0.759142,0.834565,5.225848e+08,...,-7157.441850,2.403445,7.913561,19.233593,912.914618,1170.878793,18.105264,12.720105,73.487865,1.844730
6,7.462443,0.785706,0.765963,8.560896e+06,400704.121188,9.343267,6.852462e+06,0.760987,0.833112,5.676413e+08,...,-3453.461327,2.748397,10.350442,20.725057,912.973266,1150.562966,18.275455,13.157699,74.605015,1.851460
7,7.813261,0.785515,0.766043,8.597116e+06,391942.316578,9.284504,6.825760e+06,0.762152,0.831122,5.179392e+08,...,-1442.919387,2.501529,12.454398,22.054778,913.057535,1145.712487,18.177724,13.346348,75.170150,1.905966
8,7.840669,0.788366,0.758774,8.620855e+06,389049.090551,9.247193,6.809907e+06,0.771114,0.838386,4.524144e+08,...,-3128.377228,2.073464,13.111908,22.315526,913.132268,1158.573328,18.115048,13.261122,74.043778,1.933375
9,7.553100,0.787582,0.759026,8.632052e+06,389052.079088,9.235469,6.808242e+06,0.770943,0.835784,4.213689e+08,...,-5386.337108,2.177386,9.479812,21.977949,913.165670,1160.957791,18.281050,13.470416,73.546507,1.889856


In [15]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     3092.068000
1     3749.448000
2     4174.942000
3     4522.969000
4     4578.716000
5     4341.821857
6     4108.786286
7     3869.334857
8     3764.518000
9     3773.354000
10    3786.974000
11    4235.532000
12    4784.413000
13    4538.256000
14            NaN
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [16]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.954430,-1.699678,1.331277,-1.852442,1.525597,1.304983,-1.799100,-1.621982,-0.042462,1.204752,...,-0.569731,2.026352,0.984445,-1.266771,-0.975425,1.940409,-0.268956,1.137644,1.835993,2.079287
1,1.017927,-1.261317,0.939993,-1.694326,1.677576,1.094681,-1.854414,-1.293295,0.429908,0.820657,...,-0.838980,0.085558,-0.051710,-1.745665,-1.582298,1.579454,-0.539264,1.176616,1.956737,1.461930
2,0.939098,-0.873574,0.655199,-1.104775,1.084964,0.895963,-0.871808,-0.983091,0.680031,0.275392,...,-1.042057,-1.114490,-0.115145,-1.071300,-1.521642,0.594192,-1.725446,-0.577589,0.854816,0.988761
3,0.910963,-0.535386,0.474312,-0.560051,0.614372,0.794584,-0.061186,-0.692424,0.712066,-0.132596,...,-1.062386,-0.697344,0.633403,-0.684027,-0.957222,0.107944,-1.694185,-1.555909,-0.114802,0.618558
4,-2.415320,-0.422527,0.325283,-0.050154,0.319763,0.747334,0.727969,-0.472052,0.483303,-0.104860,...,-0.940648,-0.337667,-0.646476,-0.341164,-0.385670,-0.139500,-0.919352,-1.603519,-0.544022,-0.159410
5,-1.251457,0.163252,0.508405,0.382937,0.024542,0.585657,1.261692,-0.119710,0.133711,0.657046,...,-0.446625,0.422337,-0.757614,-0.381577,0.134391,-0.373874,0.170234,-1.319280,-0.759813,-0.892823
6,-0.278927,0.325010,0.614316,0.693077,-0.216274,0.203003,1.333947,0.085716,-0.443735,1.615453,...,0.358807,1.613108,0.345772,0.406656,0.358905,-1.222272,0.973093,-0.167721,-0.121014,-0.841773
7,0.478169,0.216545,0.636582,0.807496,-0.923301,-0.593627,0.680859,0.215483,-1.234902,0.558230,...,0.796000,0.760923,1.298413,1.109410,0.681498,-1.424830,0.512055,0.328724,0.202137,-0.428361
8,0.537319,1.834498,-1.380170,0.882491,-1.156767,-1.099436,0.293116,1.213246,1.652199,-0.835563,...,0.429496,-0.716758,1.596124,1.247214,0.967586,-0.887756,0.216390,0.104443,-0.441935,-0.220470
9,-0.083280,1.389158,-1.310222,0.917863,-1.156526,-1.258364,0.252392,1.194161,0.618336,-1.495939,...,-0.061498,-0.358019,-0.048438,1.068806,1.095455,-0.788179,0.999483,0.655217,-0.726280,-0.550555


In [17]:
reshaped_train_input = dfToInputRNN(train_input)

In [18]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[-0.42881683,  0.05023865, -1.29053308,  0.69018756,
         -0.77950283, -1.23123715, -0.10251621,  1.11506099,
         -1.3313138 , -1.0453709 ,  1.21859064, -0.99976018,
          0.79428371,  1.15575749, -0.52559858,  1.30260559,
         -0.55253046, -0.25130359,  1.38787176,  1.47154451,
         -1.80649616,  1.09296291,  0.94281138,  0.52836084,
          0.61258103,  0.87656257,  1.47801536,  1.71982086,
          1.28221181,  1.39086472,  1.58209844,  1.08872091,
          0.55212962, -1.81398118,  1.44113674,  1.11191094,
          1.30470835,  0.86390516, -1.67365968, -1.70738939,
         -1.55813196, -0.48498301, -0.75714639,  0.40770893,
         -0.27394734,  2.02323219,  1.2908951 ,  1.70465513,
          1.81148084,  1.39968653, -1.36191442,  1.28024593,
          1.11054795,  0.12662799,  1.40414112,  1.40468999,
          1.45643959,  1.58509336,  1.32283157,  1.30222047,
          1.55138602,  1.91801602, -0.77489784,  2.32098884,
         -1.1942541 , -1

In [19]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     3092.068000
1     3749.448000
2     4174.942000
3     4522.969000
4     4578.716000
5     4341.821857
6     4108.786286
7     3869.334857
8     3764.518000
9     3773.354000
10    3786.974000
11    4235.532000
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [20]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    4235.532
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [21]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-0.734266,-1.069894,-1.42466,0.401023,-0.568106,-1.099064,-0.414155,1.035451,-2.637193,1.108405,...,0.964765,-1.482779,-2.260447,0.964616,0.339314,-0.540552,0.224215,0.839814,-2.237536,-1.155068


In [22]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[-0.73426567, -1.06989407, -1.42465965,  0.40102323,
         -0.56810629, -1.099064  , -0.41415465,  1.03545073,
         -2.63719315,  1.10840491,  2.63582309, -0.18995911,
          0.72640489,  2.66833308, -0.032632  ,  0.98802435,
          0.24980494, -0.08955808,  1.85434394,  2.02706568,
         -2.21150703,  1.42938591,  0.85132668,  0.14092386,
          0.30902324,  1.00267904,  2.27970137,  1.11272593,
          0.61007902,  0.89064538,  1.42231438,  1.02650562,
          1.47127593, -1.53201473,  1.73503858,  1.66610613,
          1.0456894 ,  0.55289259, -1.81216593, -2.10342296,
         -2.55377478,  0.41940628, -1.75756445,  0.241023  ,
         -0.98659345,  1.52184916,  1.01398607,  1.49290967,
          1.40764092,  1.87930411, -2.51892072,  2.83356214,
          1.58620924, -0.02524128,  1.54231542,  1.5176119 ,
          1.7355269 ,  1.87941982,  1.54198129,  1.51276712,
          1.83795594,  2.62210693,  1.04993174,  0.96476493,
         -1.48277861, -2

In [23]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    4784.413
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [36]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [37]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [38]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3730141063, 940192249, 54631859, 4161008103, 3500505539, 464639109, 110240047, 4042704893, 4280994417, 2247166427, 2730299296, 1766653291, 1903013116, 430633960, 910527912, 2820224395, 483969332, 3570636968, 2840894666, 3592675070, 2653133453, 742831225, 155345721, 683963482, 2389084750, 64313727, 2846203277, 234859283, 2240302954, 24310006, 1287351210, 1259685116, 3216873157, 879234067, 307309010, 2852058581, 1051285806, 2758254188, 971654498, 2118547162, 1416178981, 3683329256, 3383518923, 2819312003, 3162904778, 1857845972, 1008264996, 1617586321, 2470303953, 4229308736, 219908255, 3423663312, 3796993340, 278794042, 751215504, 310289904, 3992936483, 2615530622, 95238523, 2013870366, 317355817, 2643140572, 2043552520, 3932319400, 2092643445, 4066618669, 4192430160, 450693628, 1264695860, 1386690722, 1963351017, 3529210720, 1455728528, 3330617110, 1817317317, 847234741, 3191150352, 1919890726, 2380512346, 3288388899, 2123691142, 1487699878, 2192393854, 983998459, 2382235938, 44208825

loss: 49577.42578125


Step: 93 ___________________________________________
loss: 85522.0859375


Step: 94 ___________________________________________
loss: 68463.2734375


Step: 95 ___________________________________________
loss: 84499.8359375


Step: 96 ___________________________________________
loss: 42247.9375


Step: 97 ___________________________________________
loss: 54499.70703125


Step: 98 ___________________________________________
loss: 37394.64453125


Step: 99 ___________________________________________
loss: 55049.66015625


final_seed: 3570636968


In [39]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 4s 4s/step - loss: 16155472.0000
Epoch 2/10000
1/1 [==============================] - 0s 8ms/step - loss: 15453989.0000
Epoch 3/10000
1/1 [==============================] - 0s 8ms/step - loss: 4087759.2500
Epoch 4/10000
1/1 [==============================] - 0s 8ms/step - loss: 1981204.0000
Epoch 5/10000
1/1 [==============================] - 0s 5ms/step - loss: 3622426.0000
Epoch 6/10000
1/1 [==============================] - 0s 8ms/step - loss: 930095.4375
Epoch 7/10000
1/1 [==============================] - 0s 5ms/step - loss: 1022088.6875
Epoch 8/10000
1/1 [==============================] - 0s 7ms/step - loss: 238565.8594
Epoch 9/10000
1/1 [==============================] - 0s 5ms/step - loss: 279116.7812
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 287957.4688
Epoch 11/10000
1/1 [==============================] - 0s 5ms/step - loss: 567404.0000
Epoch 12/10000
1/1 [==============================] - 0s

1/1 [==============================] - 0s 4ms/step - loss: 170917.1875
Epoch 97/10000
1/1 [==============================] - 0s 5ms/step - loss: 182260.9219
Epoch 98/10000
1/1 [==============================] - 0s 6ms/step - loss: 374853.8438
Epoch 99/10000
1/1 [==============================] - 0s 5ms/step - loss: 172270.0781
Epoch 100/10000
1/1 [==============================] - 0s 5ms/step - loss: 192491.6719
Epoch 101/10000
1/1 [==============================] - 0s 6ms/step - loss: 205519.7031
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 314772.5938
Epoch 103/10000
1/1 [==============================] - 0s 5ms/step - loss: 223675.7969
Epoch 104/10000
1/1 [==============================] - 0s 5ms/step - loss: 196791.5781
Epoch 105/10000
1/1 [==============================] - 0s 5ms/step - loss: 336404.0938
Epoch 106/10000
1/1 [==============================] - 0s 5ms/step - loss: 106643.9141
Epoch 107/10000
1/1 [==============================] - 0s 5ms/

1/1 [==============================] - 0s 5ms/step - loss: 157882.2344
Epoch 191/10000
1/1 [==============================] - 0s 5ms/step - loss: 257143.3594
Epoch 192/10000
1/1 [==============================] - 0s 4ms/step - loss: 253293.5469
Epoch 193/10000
1/1 [==============================] - 0s 5ms/step - loss: 148439.7500
Epoch 194/10000
1/1 [==============================] - 0s 5ms/step - loss: 127546.6641
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 307305.5312
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 187901.2656
Epoch 197/10000
1/1 [==============================] - 0s 5ms/step - loss: 227342.0469
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 155980.9219
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 74965.5234
Epoch 200/10000
1/1 [==============================] - 0s 5ms/step - loss: 238155.6719
Epoch 201/10000
1/1 [==============================] - 0s 4m

1/1 [==============================] - 0s 4ms/step - loss: 363505.6562
Epoch 285/10000
1/1 [==============================] - 0s 5ms/step - loss: 171198.2969
Epoch 286/10000
1/1 [==============================] - 0s 9ms/step - loss: 53190.1562
Epoch 287/10000
1/1 [==============================] - 0s 5ms/step - loss: 196395.7031
Epoch 288/10000
1/1 [==============================] - 0s 6ms/step - loss: 139509.5156
Epoch 289/10000
1/1 [==============================] - 0s 4ms/step - loss: 156869.7500
Epoch 290/10000
1/1 [==============================] - 0s 5ms/step - loss: 233899.2656
Epoch 291/10000
1/1 [==============================] - 0s 6ms/step - loss: 214944.5469
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 59483.7617
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 177530.7969
Epoch 294/10000
1/1 [==============================] - 0s 6ms/step - loss: 110167.5547
Epoch 295/10000
1/1 [==============================] - 0s 4ms

1/1 [==============================] - 0s 4ms/step - loss: 200850.6875
Epoch 379/10000
1/1 [==============================] - 0s 4ms/step - loss: 171962.2188
Epoch 380/10000
1/1 [==============================] - 0s 6ms/step - loss: 121562.1016
Epoch 381/10000
1/1 [==============================] - 0s 4ms/step - loss: 104290.9375
Epoch 382/10000
1/1 [==============================] - 0s 4ms/step - loss: 136875.6719
Epoch 383/10000
1/1 [==============================] - 0s 6ms/step - loss: 115424.0078
Epoch 384/10000
1/1 [==============================] - 0s 4ms/step - loss: 113772.1562
Epoch 385/10000
1/1 [==============================] - 0s 4ms/step - loss: 175066.2500
Epoch 386/10000
1/1 [==============================] - 0s 5ms/step - loss: 213310.3281
Epoch 387/10000
1/1 [==============================] - 0s 5ms/step - loss: 359189.5312
Epoch 388/10000
1/1 [==============================] - 0s 5ms/step - loss: 164487.2188
Epoch 389/10000
1/1 [==============================] - 0s 5

...

1/1 [==============================] - 0s 6ms/step - loss: 164527.4219
Epoch 661/10000
1/1 [==============================] - 0s 4ms/step - loss: 150996.4219
Epoch 662/10000
1/1 [==============================] - 0s 4ms/step - loss: 135203.2812
Epoch 663/10000
1/1 [==============================] - 0s 6ms/step - loss: 180429.3125
Epoch 664/10000
1/1 [==============================] - 0s 4ms/step - loss: 184530.2969
Epoch 665/10000
1/1 [==============================] - 0s 5ms/step - loss: 186809.2031
Epoch 666/10000
1/1 [==============================] - 0s 5ms/step - loss: 137479.2969
Epoch 667/10000
1/1 [==============================] - 0s 5ms/step - loss: 138231.2344
Epoch 668/10000
1/1 [==============================] - 0s 4ms/step - loss: 197561.5000
Epoch 669/10000
1/1 [==============================] - 0s 5ms/step - loss: 218300.4531
Epoch 670/10000
1/1 [==============================] - 0s 5ms/step - loss: 199521.9219
Epoch 671/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 4ms/step - loss: 104991.5234
Epoch 755/10000
1/1 [==============================] - 0s 5ms/step - loss: 295112.5938
Epoch 756/10000
1/1 [==============================] - 0s 5ms/step - loss: 148856.9531
Epoch 757/10000
1/1 [==============================] - 0s 4ms/step - loss: 139414.0000
Epoch 758/10000
1/1 [==============================] - 0s 4ms/step - loss: 94291.8984
Epoch 759/10000
1/1 [==============================] - 0s 5ms/step - loss: 216780.5156
Epoch 760/10000
1/1 [==============================] - 0s 4ms/step - loss: 155531.1875
Epoch 761/10000
1/1 [==============================] - 0s 4ms/step - loss: 187129.6719
Epoch 762/10000
1/1 [==============================] - 0s 5ms/step - loss: 130751.2500
Epoch 763/10000
1/1 [==============================] - 0s 5ms/step - loss: 167208.8125
Epoch 764/10000
1/1 [==============================] - 0s 5ms/step - loss: 125275.9453
Epoch 765/10000
1/1 [==============================] - 0s 5m

1/1 [==============================] - 0s 4ms/step - loss: 203787.1094
Epoch 849/10000
1/1 [==============================] - 0s 6ms/step - loss: 172695.6406
Epoch 850/10000
1/1 [==============================] - 0s 4ms/step - loss: 305848.7188
Epoch 851/10000
1/1 [==============================] - 0s 4ms/step - loss: 143018.5000
Epoch 852/10000
1/1 [==============================] - 0s 5ms/step - loss: 211254.9375
Epoch 853/10000
1/1 [==============================] - 0s 5ms/step - loss: 83139.2578
Epoch 854/10000
1/1 [==============================] - 0s 5ms/step - loss: 95369.0000
Epoch 855/10000
1/1 [==============================] - 0s 4ms/step - loss: 147093.8281
Epoch 856/10000
1/1 [==============================] - 0s 4ms/step - loss: 112115.5859
Epoch 857/10000
1/1 [==============================] - 0s 4ms/step - loss: 110292.9062
Epoch 858/10000
1/1 [==============================] - 0s 5ms/step - loss: 145862.7656
Epoch 859/10000
1/1 [==============================] - 0s 4ms

1/1 [==============================] - 0s 5ms/step - loss: 52135.1367
Epoch 943/10000
1/1 [==============================] - 0s 5ms/step - loss: 247949.0469
Epoch 944/10000
1/1 [==============================] - 0s 4ms/step - loss: 186497.5625
Epoch 945/10000
1/1 [==============================] - 0s 4ms/step - loss: 213908.2969
Epoch 946/10000
1/1 [==============================] - 0s 5ms/step - loss: 224406.7031
Epoch 947/10000
1/1 [==============================] - 0s 5ms/step - loss: 80567.8750
Epoch 948/10000
1/1 [==============================] - 0s 6ms/step - loss: 138317.2344
Epoch 949/10000
1/1 [==============================] - 0s 5ms/step - loss: 295386.2188
Epoch 950/10000
1/1 [==============================] - 0s 5ms/step - loss: 73421.1406
Epoch 951/10000
1/1 [==============================] - 0s 6ms/step - loss: 162687.8594
Epoch 952/10000
1/1 [==============================] - 0s 4ms/step - loss: 142847.9844
Epoch 953/10000
1/1 [==============================] - 0s 5ms/

1/1 [==============================] - 0s 6ms/step - loss: 218947.1875
Epoch 1037/10000
1/1 [==============================] - 0s 5ms/step - loss: 161012.5781
Epoch 1038/10000
1/1 [==============================] - 0s 4ms/step - loss: 226010.6875
Epoch 1039/10000
1/1 [==============================] - 0s 5ms/step - loss: 155468.8906
Epoch 1040/10000
1/1 [==============================] - 0s 5ms/step - loss: 133967.3594
Epoch 1041/10000
1/1 [==============================] - 0s 5ms/step - loss: 187728.3906
Epoch 1042/10000
1/1 [==============================] - 0s 5ms/step - loss: 78483.5547
Epoch 1043/10000
1/1 [==============================] - 0s 4ms/step - loss: 215743.3281
Epoch 1044/10000
1/1 [==============================] - 0s 4ms/step - loss: 198850.2969
Epoch 1045/10000
1/1 [==============================] - 0s 6ms/step - loss: 283754.3438
Epoch 1046/10000
1/1 [==============================] - 0s 5ms/step - loss: 127969.5391
Epoch 1047/10000
1/1 [============================

In [40]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 655ms/step
[4784.413](test_target) - [[3880.3655]](prediction) = 904.0475214843746


In [41]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.18895683158715074

In [42]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [43]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.470460,-1.248978,1.283752,-0.938459,0.382853,1.236143,-0.644859,-1.236383,-1.329522,1.148205,...,1.277766,1.308949,1.412346,0.333480,1.408380,1.000190,0.909918,0.683149,0.581225,1.275014
1,1.390219,0.049998,-0.128080,-0.446997,0.987585,-0.023123,-0.767579,0.023618,0.247295,0.140893,...,-0.114016,-0.190804,-0.643250,-1.356947,-0.815318,0.365766,0.482610,0.730797,0.825914,-0.107644
2,-0.919759,1.198980,-1.155672,1.385456,-1.370438,-1.213019,1.412438,1.212765,1.082228,-1.289098,...,-1.163750,-1.118144,-0.769096,1.023467,-0.593062,-1.365956,-1.392529,-1.413946,-1.407140,-1.167370


train_target:


,Paraná - Consumo de Cimento (t)
0,3092.068
1,3749.448
2,4174.942


test_input:


...

,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-0.734266,-1.069894,-1.42466,0.401023,-0.568106,-1.099064,-0.414155,1.035451,-2.637193,1.108405,...,0.964765,-1.482779,-2.260447,0.964616,0.339314,-0.540552,0.224215,0.839814,-2.237536,-1.155068


test_target:


,Paraná - Consumo de Cimento (t)
12,4784.413


1/1 [==============================] - 1s 603ms/step
Error: 904.0475214843746




[4067.547607421875,
 4308.02978515625,
 3628.83251953125,
 4215.1767578125,
 3783.094970703125,
 3896.31982421875,
 3719.881591796875,
 3958.10693359375,
 3817.26708984375,
 3880.365478515625]

In [44]:
display(targets)
display(predictions)

[4522.969,
 4578.716,
 4341.821857142857,
 4108.786285714286,
 3869.334857142857,
 3764.518,
 3773.354,
 3786.974,
 4235.532,
 4784.413]

[4067.547607421875,
 4308.02978515625,
 3628.83251953125,
 4215.1767578125,
 3783.094970703125,
 3896.31982421875,
 3719.881591796875,
 3958.10693359375,
 3817.26708984375,
 3880.365478515625]

In [45]:
mae = mean_absolute_error(predictions, targets)
mae

331.04469012276775

In [46]:
porcentage = mae/np.mean(targets)
porcentage

0.07926097042764613